# Lab Assignment Three: Extending Logistic Regression

## ***** Catherine Magee, Morgan Mote, Luv Patel *****

### In this lab, you will compare the performance of logistic regression optimization programmed in scikit-learn and via your own implementation. You will also modify the optimization procedure for logistic regression. 

This report is worth 10% of the final grade. Please upload a report (one per team) with all code used, visualizations, and text in a rendered Jupyter notebook. Any visualizations that cannot be embedded in the notebook, please provide screenshots of the output. The results should be reproducible using your report. Please carefully describe every assumption and every step in your report.

## Dataset Selection

Select a dataset identically to the way you selected for the lab one (i.e., table data and you must have a mix of numeric and categorical variables). You are not required to use the same dataset that you used in the past, but you are encouraged. You must identify a classification task from the dataset that contains three or more classes to predict. That is it cannot be a binary classification; it must be multi-class prediction. 

## Grading Rubric

### Preparation and Overview (3 points total)
- [2 points] Explain the task and what business-case or use-case it is designed to solve (or designed to investigate). Detail exactly what the classification task is and what parties would be interested in the results. For example, would the model be deployed or used mostly for offline analysis? As in previous labs, also detail how good the classifier needs to perform in order to be useful. 
- [.5 points] (mostly the same processes as from previous labs) Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis. Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created). Provide a breakdown of the variables after preprocessing (such as the mean, std, etc. for all variables, including numeric and categorical). 
- [.5 points] Divide your data into training and testing data using an 80% training and 20% testing split. Use the cross validation modules that are part of scikit-learn. Argue "for" or "against" splitting your data using an 80/20 split. That is, why is the 80/20 split appropriate (or not) for your dataset?  

### Modeling (5 points total)
- The implementation of logistic regression must be written only from the examples given to you by the instructor. No credit will be assigned to teams that copy implementations from another source, regardless of if the code is properly cited. 
- [2 points] Create a custom, one-versus-all logistic regression classifier using numpy and scipy to optimize. Use object oriented conventions identical to scikit-learn. You should start with the template developed by the instructor in the course. You should add the following functionality to the logistic regression classifier:
  - Ability to choose optimization technique when class is instantiated: either steepest ascent, stochastic gradient ascent, and {Newton's method/Quasi Newton methods}. 
  - Update the gradient calculation to include a customizable regularization term (either using no regularization, L1 regularization, L2 regularization, or both L1 and L2 regularization). Associate a cost with the regularization term, "C", that can be adjusted when the class is instantiated.  
- [1.5 points] Train your classifier to achieve good generalization performance. That is, adjust the optimization technique and the value of the regularization term(s) "C" to achieve the best performance on your test set. Visualize the performance of the classifier versus the parameters you investigated. Is your method of selecting parameters justified? That is, do you think there is any "data snooping" involved with this method of selecting parameters?
- [1.5 points] Compare the performance of your "best" logistic regression optimization procedure to the procedure used in scikit-learn. Visualize the performance differences in terms of training time and classification performance. Discuss the results. 

### Deployment (1 points total)
- Which implementation of logistic regression would you advise be used in a deployed machine learning model, your implementation or scikit-learn (or other third party)? Why?

### Exceptional Work (1 points total)
- You have free reign to provide additional analyses. One idea: Update the code to use either "one-versus-all" or "one-versus-one" extensions of binary to multi-class classification. 

- Required for 7000 level students: Choose ONE of the following:

  - Option One: Implement an optimization technique for logistic regression using mean square error as your objective function (instead of maximum likelihood). Derive the gradient updates for the Hessian and use Newton's method to update the values of "w". Then answer, which process do you prefer: maximum likelihood OR minimum mean-squared error? 
  - Option Two: Implement the BFGS algorithm from scratch to optimize logistic regression. That is, use BFGS without the use of an external package (for example, do not use SciPy). Compare your performance accuracy and runtime to the BFGS implementation in SciPy (that we used in lecture). 

In [ ]:
# Standard Library Imports
from IPython.display import display, HTML, Markdown
import warnings
import textwrap
import os
import sys
import glob
import zipfile
from pathlib import Path
import platform
import subprocess
import copy
from typing import Optional

# Third-Party Imports
import pandas as pd
import numpy as np
from PIL import Image
import cv2
from tqdm import tqdm

# Plotting
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import bokeh
import missingno as mn

# Tensorflow
import tensorflow as tf
from tensorflow import keras

# Sci-py Imports
import scipy
from scipy.special import expit

# Imblearn Imports
from imblearn.over_sampling import SMOTE

# Skimage Imports
import skimage

# Sklearn Imports
import sklearn
from sklearn import metrics
from sklearn import svm
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import pairwise
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder,
    OrdinalEncoder,
    LabelEncoder,
)
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)

# Warning Filters
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="fastai")
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

#Random number generator seed
np.random.seed(123)

In [ ]:
# Dataset Import
#CSV file is in the same directory as Python script
file_path = "COVID-19_cases_plus_census.csv"

# Read the CSV file into a DataFrame
cases = pd.read_csv(file_path)

# Now, 'cases' contains the data from the CSV file

## Business Understanding and Problem Solving

Here is a detailed description of the task and what business-case or use-case it is designed to solve (or designed to investigate). We have detailed exactly what the classification task is and what parties would be interested in the results. We should consider the example, would the model be deployed or used mostly for offline analysis? As in previous labs, we also need to detail how well the classifier needs to perform in order to be useful.

We retrieved out dataset from the following source: https://console.cloud.google.com/marketplace/details/usafacts-public-data/covid19-us-cases?filter=solution-type:dataset&filter=category:covid19&id=3eaff9c5-fbaf-47bb-a441-89db1e1395ab&project=still-nebula-398202 [1]

### ***COPIED FROM FIRST PROJECT PLEASE ENHANCE ACCORDINGLY FOR THIS PROJECT***

This exploratory analysis focuses on COVID 19 data pulled from the Google Cloud Public Datasets Program. More Specifically, this dataset focuses on a census report in Texas that was collected during the pandemic in 2021. The purpose of this dataset is to discover which counties in Texas are at highest risk for positive COVID 19 cases.

This analysis is essential to flattening the curve or slowing the spread of postive COVID 19 cases because it surfaces not only the counties that are at the highest risk but also which demographic in that county needs the most assistance. Therefore this analysis can provide useful information to third parties such as hospitals in their distribution of personal protective equipment (PPE) such as masks.

The COVID-19 pandemic had a major impact on every facet of the human experience for people all over the world. Businesses closed while people lost their jobs, homes, and loved ones. Through the significant human impact, data collection has allowed analysts to shed light on exactly how the pandemic affected businesses and economies worldwide. On a regulatory front, there were many inconsistencies in public health regulations and guidelines between federal and state agencies. These disparities fueled public health concerns and debates which contributed to an increased sense of confusion and lack of effectiveness of the measures put in place to protect the public from COVID-19 infection. The data was collected to provide an accurate view of the affects of a pandemic in the US so that we may be better prepared to face a pandemic in the future.

Through our analysis, we have revealed that some areas of the country have less deaths overall but higher mortality rates from confirmed COVID-19 cases due to many factors such as local population size and density, proximity to an urban area, and many other aspects will be explored. Some areas with a low population have experienced a higher deaths-per-thousand count than large urban areas with a higher number of deaths but a lower count of deaths-per-thousand. The goal of this analysis is to predict which counties will be at the highest risk with the lowest amount of resources in the future. Based on these notions and within a certain degree of accuracy, it may be possible to predict the development in a region given comparable data from other regions.

# Data Preparation

### Feature Evaluation

In [ ]:
def print_df_list_of_features():
    columns = df.columns.to_list()
    features = "Features: "
    for i, col in enumerate(columns):
        if i > 0:
            features += f', "{col}"'
        else:
            features += f'"{col}"'
    print(features)

print_df_list_of_features()

In [ ]:
def print_df_shape():
    rows, cols = df.shape
    print(f"~ Dataframe Size ~\nRows: {rows}\nCoumns: {cols}")

print_df_shape()

In [ ]:
# Filter rows with confirmed_cases > 0
cases = cases[cases['confirmed_cases'] > 0]

# Calculate rates per 1000 people 
# Rate calculation added as new attributes
cases['cases_per_1000'] = (cases['confirmed_cases'] / cases['total_pop']) * 1000
cases['deaths_per_1000'] = (cases['deaths'] / cases['total_pop']) * 1000
cases['death_per_case'] = cases['deaths'] / cases['confirmed_cases']

# Sort the DataFrame by confirmed_cases in descending order
cases = cases.sort_values(by='confirmed_cases', ascending=False)

### Prepare Feature Categories

In [ ]:
# Select important attributes
selected_columns = ['county_name', 'state', 'confirmed_cases', 'deaths', 'total_pop', 'median_income', 
                    'median_age', 'cases_per_1000', 'deaths_per_1000', 'death_per_case']
cases = cases[selected_columns]

In [ ]:
def get_feature_col_counts():
    cols = ['county_name', 'state', 'confirmed_cases', 'deaths', 'total_pop', 'median_income', 
                    'median_age', 'cases_per_1000', 'deaths_per_1000', 'death_per_case']
    print(f"{' Value Counts '.center(40, '*')}\n")
    for col in cols:
        print(df[col].value_counts(), "\n")
    print(f"{''.center(40, '*')}")

get_feature_col_counts()

In [ ]:
# Print the column names in the 'cases' DataFrame
print("These are the cases in the dataframe. \n" + str(cases.columns)+ "\n" + "\n")

# Print the first few rows in the resulting DataFrame
print(cases.head())

### Validating the Dataset

In [ ]:
# Check for missing values
missing_values = cases.isnull().sum()
print("Missing Values:")
print(str(missing_values)+ "\n")

print("These are the columns with missing data:" + str(cases.columns[cases.isnull().any()].tolist()))

In [ ]:
# Check for duplicate data
def check_for_duplicate_entries():
    return df.duplicated().any()


print(
    "Duplicates Found"
) if check_for_duplicate_entries() is True else print("No Duplicates Found")

In [ ]:
# Visualize missing data
def visualize_missing_data(title):
    mn.matrix(df.sort_values(by=[" ########## "]), color=(.4, .75, 1)) # replace with category from our data set
    plt.title(title, fontsize=20)
    plt.show()

visualize_missing_data("Missing Data Visualized (Before Imputation)")

In [ ]:
# Impute missing values using K-Nearest Neighbor for the column BMI
imputer = KNNImputer(n_neighbors=5, weights="uniform")
bmi_imputed_vals = imputer.fit_transform(df[" ########## "].to_numpy().reshape(-1, 1)) # replace with same category from above fuction
df[" ########## "] = bmi_imputed_vals # replace with same category from above fuction

# Show that missing values were imputed
visualize_missing_data("Missing Data Visualized (After Imputation)")

### Identify Categorical Feature Variables

In [ ]:
df.describe(exclude=[np.number])

In [ ]:
def unique_feature_categories():
    print("%s\n" % format(" Unique Categories ".center(50, "*")))

    selected_cols = df.select_dtypes(exclude=[np.number]).columns.to_list()
    selected_cols = [c for c in selected_cols]

    for col in selected_cols:
        if not isinstance(df[col].values, float):
            unique_vals = pd.unique(df[col].values)
            print(
                col, ', '.join([v for v in unique_vals if not isinstance(v, float)]),
                sep=": "
            )
            print()

    print("%s" % format("".center(50, "*")))

unique_feature_categories()

### Identify Continuous Feature Variables

In [ ]:
df[ "median_age", "confirmed_cases", "deaths_per_case"].describe(include=[np.number]).round(1) 

### View Data Based on Gender and Age

In [ ]:
def group_gender_and_age():
    age = df["age"].values
    bins = [age for age in range(0, 100, 10)]  # Generator for [0, ..., 90]

    age_binned = pd.cut(age, bins, labels=False)

    tmp_df = df.copy()
    tmp_df["age group"] = age_binned

    tmp_df.rename(
        columns={
            "median_age": "median age",
            "total_pop": "total population",
            "confirmed_cases": "confirmed cases",
            "median_income": "median income",
            "deaths_per_1000": "deaths per 1000"
        },
        inplace=True,
    )

    grouped = (
        tmp_df.groupby(["gender", "age group"])
        .agg(
            {
                "median age": "mean",
                "total population": "sum",
                "confirmed cases": "sum",
                "median income": "mean",
                "deaths per 1000": "sum",
            }
        )
        .round(1)
    )
    display(HTML(grouped.to_html()))


group_gender_and_age()

In [ ]:
def group_gender_counts():
    gender_counts = df["gender"].value_counts().to_dict()
    tmp_df = pd.DataFrame.from_dict(gender_counts, orient='index', columns=['count'])
    tmp_df = tmp_df.rename({0: "Male", 1: "Female"})
    display(HTML(tmp_df.to_html()))

group_gender_counts()

In [ ]:
df[["gender", "deaths_per_case"]].groupby("gender").sum()

In [ ]:
df.groupby('deaths_per_case').count()

### Define Target Features

In [ ]:
# Separate the target variable from the rest of the features
target = df["deaths_per_case"]
features = df.drop("deaths_per_case", axis=1)

continuous_features = [
    "median_age",
    "confirmed_cases",
    "deaths_per_case",
]
categorical_features = [
    " aaaa ",
    " bbbb ",
    " cccc ",
    " dddd ",
    " eeee ",
    " ffff ", # replace with actual category features
]

### Normalize Continuous Features

In [ ]:
scaler = StandardScaler()

In [ ]:
features[continuous_features] = scaler.fit_transform(features[continuous_features])

### Handling Class Imbalance

In [ ]:
plt.bar([0, 1], np.bincount(target))
plt.xticks([0, 1], ['No Confirmation', 'Confirmed'])
plt.title('Class Imbalance For COVID-19 Confirmed Cases Data') 
plt.show()

# please change the attribute for the class imbalance section if it doesnt work

In [ ]:
def resample_for_class_imbalance():
    """
    Use SMOTE for balancing the dataset to deal with the relatively
    small number of unconfirmed cases. IF THERE EVEN ARE ANY, MIGHT NEED TO SELECT A DIFFERENT ATTRIBUTE!!!***************
    """
    encoder = LabelEncoder()
    features[categorical_features] = features[categorical_features].apply(
        lambda ftr: encoder.fit_transform(ftr)
    )
    oversample_smote = SMOTE(random_state=123)
    x_resample, y_resample = oversample_smote.fit_resample(features, target)
    return x_resample, y_resample


x_resample, y_resample = resample_for_class_imbalance()

In [ ]:
plt.bar([0, 1], np.bincount(y_resample))
plt.xticks([0, 1], ['No Confirmation', 'Confirmed'])
plt.title("Class Balance After Using SMOTE")
plt.show()

In [ ]:
x_resample.shape, y_resample.shape

### View Finalized Preprocessed Dataframe

In [ ]:
x_resample.describe().round(2)

In [ ]:
y_resample.describe().round(2)

### Split Into Training and Testing Data

In [ ]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
    # features, target
    x_resample,
    y_resample,
    test_size=0.2,
    random_state=123, # refer to import cell
    stratify=y_resample,
)

# Modeling

### Initial Logistic Regression Using Sklearn 

[2] https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
# Combine the preprocessor and the linear regression model into a pipeline
pipeline = Pipeline(
    steps=[("regressor", LogisticRegression())]
)
pipeline.fit(x_train, y_train)
# Make predictions on the testing data
y_pred = pipeline.predict(x_test)

In [ ]:
# Evaluate the performance of the model on the testing data
score = pipeline.score(x_test, y_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(confusion)

### Binary Logistic Regression

[3] https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8

In [ ]:
Binary Logistic Regression
class BinaryLogisticRegressionBase:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with
        # sklearn conventions

    def __str__(self):
        return "Base Binary Logistic Regression Object, Not Trainable"

    # convenience, private and static:
    @staticmethod
    def _sigmoid(theta):
        return 1 / (1 + np.exp(-theta))

    @staticmethod
    def _add_intercept(X):
        return np.hstack((np.ones((X.shape[0], 1)), X))  # add bias term
    
    def _add_bias(self, X):
        return self._add_intercept(X)

    def predict_proba(self, X, add_intercept=True):
        # add bias term if requested
        Xb = self._add_intercept(X) if add_intercept else X
        return self._sigmoid(Xb @ self.w_)  # return the probability y=1

    def predict(self, X):
        return self.predict_proba(X) > 0.5  # return the actual prediction

In [ ]:
class BinaryLogisticRegression(BinaryLogisticRegressionBase):
    def __str__(self):
        if hasattr(self, "w_"):
            return (
                "Binary Logistic Regression Object with coefficients:\n"
                + str(self.w_)
            )  # is we have trained the object
        else:
            return "Untrained Binary Logistic Regression Object"

    def _get_gradient(self, X, y):
        # programming \sum_i (yi-g(xi))xi
        gradient = np.zeros(self.w_.shape)  # set gradient to zero
        for (xi, yi) in zip(X, y):
            # the actual update inside of sum
            gradi = (yi - self.predict_proba(xi, add_intercept=False)) * xi
            # reshape to be column vector and add to gradient
            gradient += gradi.reshape(self.w_.shape)

        return gradient / float(len(y))

    # public:
    def fit(self, X, y):
        Xb = self._add_intercept(X)  # add bias term
        num_samples, num_features = Xb.shape

        # init weight vector to zeros
        self.w_ = np.zeros((num_features, 1))

        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb, y)
            self.w_ += gradient * self.eta  # multiply by learning rate

In [ ]:
def run_binary_logistic_regression(**kwargs):
    blr = BinaryLogisticRegression(**kwargs)
    blr.fit(x_train, y_train)
    y_pred = blr.predict(x_test)
    
    print(f"Shape: {blr.w_.shape}")
    print(blr)
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

run_binary_logistic_regression(eta=0.1, iterations=24)
run_binary_logistic_regression(eta=0.1, iterations=700)

### Line Search Logistic Regression

[4] https://github.com/eclarson/MachineLearningNotebooks/blob/master/06.%20Optimization.ipynb

In [ ]:
class LineSearchLogisticRegression(BinaryLogisticRegression):

    # define custom line search for problem
    def __init__(self, line_iters=0.0, C=0.005, **kwargs):
        self.line_iters = line_iters
        self.C = C
        # but keep other keywords
        super().__init__(**kwargs)  # call parent initializer

    # this defines the function with the first input to be optimized
    # therefore eta will be optimized, with all inputs constant
    @staticmethod
    def objective_function(eta, X, y, w, grad, C):
        wnew = w - grad * eta
        g = expit(X @ wnew)
        # the line search is looking for minimization, so take the negative of l(w)
        return (
            -np.sum(np.ma.log(g[y == 1]))
            - np.ma.sum(np.log(1 - g[y == 0]))
            + C * sum(wnew**2)
        )


    def fit(self, X, y):
        Xb = self._add_bias(X)  # add bias term
        num_samples, num_features = Xb.shape

        self.w_ = np.zeros((num_features, 1))  # init weight vector to zeros

        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = -self._get_gradient(Xb, y)
            # minimization is in opposite direction

            # do line search in gradient direction, using scipy function
            opts = {"maxiter": self.line_iters}  # unclear exactly what this should be
            res = scipy.optimize.minimize_scalar(
                self.objective_function,  # objective function to optimize
                bounds=(0, self.eta * 10),  # bounds to optimize
                args=(
                    Xb,
                    y,
                    self.w_,
                    gradient,
                    self.C,
                ),  # additional argument for objective function
                method="bounded",  # bounded optimization for speed
                options=opts,
            )  # set max iterations

            eta = res.x  # get optimal learning rate
            self.w_ -= gradient * eta  # set new function values
            # subtract to minimize

In [ ]:
def run_line_search_logistic_regression(**kwargs):
    lslr = LineSearchLogisticRegression(**kwargs)
    lslr.fit(x_train, y_train)
    y_pred = lslr.predict(x_test)
    
    print(f"Shape: {lslr.w_.shape}")
    print(lslr)
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

run_line_search_logistic_regression(eta=0.1, iterations=24, C=0.005)

### Vectorized Binary Logistic Regression

In [ ]:
class VectorBinaryLogisticRegression(BinaryLogisticRegression):
    def __str__(self):
        if hasattr(self, "w_"):
            return (
                "Vector Binary Logistic Regression Object with coefficients:\n"
                + str(self.w_)
            )  # is we have trained the object
        else:
            return "Untrained Vector Binary Logistic Regression Object"

    # inherit from our previous class to get same functionality
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta)  # 1/(1+np.exp(-theta)) 

    # but overwrite the gradient calculation
    def _get_gradient(self, X, y):
        ydiff = (
            y - self.predict_proba(X, add_intercept=False).ravel()
        )  # get y difference
        gradient = np.mean(
            X * ydiff[:, np.newaxis], axis=0
        )  # make ydiff a column vector and multiply through

        return gradient.reshape(self.w_.shape)

In [ ]:
def run_vectorized_logistic_regression(**kwargs):
    vlr = VectorBinaryLogisticRegression(**kwargs)
    vlr.fit(x_train, y_train)
    y_pred = vlr.predict(x_test)
    
    print(f"Shape: {vlr.w_.shape}")
    print(vlr)
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

In [ ]:
run_vectorized_logistic_regression(eta=0.1, iterations=24)

In [ ]:
run_vectorized_logistic_regression(eta=0.1, iterations=1000)

### Multi-Class (One-Versus-All) Logistic Regression

[5] https://github.com/mGalarnyk/DSGO_IntroductionScikitLearn/blob/main/notebooks/LogisticOneVsAll.ipynb

In [ ]:
class MCLogisticRegression:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions

    def __str__(self):
        if hasattr(self, "w_"):
            return (
                "MultiClass Logistic Regression Object with coefficients:\n"
                + str(self.w_)
            )  # is we have trained the object
        else:
            return "Untrained MultiClass Logistic Regression Object"

    def fit(self, X, y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y)  # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = []  # will fill this array with binary classifiers

        for i, yval in enumerate(self.unique_):  # for each unique value
            y_binary = y == yval  # create a binary problem
            # train the binary classifier for this class
            blr = VectorBinaryLogisticRegression(self.eta, self.iters)
            blr.fit(X, y_binary)
            # add the trained classifier to the list
            self.classifiers_.append(blr)

        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T

    def predict_proba(self, X):
        probs = []
        for blr in self.classifiers_:
            probs.append(
                blr.predict_proba(X)
            )  # get probability for each classifier

        return np.hstack(probs)  # make into single matrix

    def predict(self, X):
        return self.unique_[
            np.argmax(self.predict_proba(X), axis=1)
        ]  # take argmax along row

In [ ]:
def run_multiclass_logistic_regression(**kwargs):
    mlr = MCLogisticRegression(**kwargs)
    mlr.fit(x_train, y_train)
    y_pred = mlr.predict(x_test)
    
    print(f"Shape: {mlr.w_.shape}")
    print(mlr)
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

In [ ]:
run_multiclass_logistic_regression(eta=0.1, iterations=10)

In [ ]:
run_multiclass_logistic_regression(eta=0.1, iterations=1000)

In [ ]:
def compare_own_logistic_regression_model_with_scikit_version():
    lr_model = LogisticRegression(solver='liblinear')
    lr_model.fit(x_train, y_train)
    y_pred = lr_model.predict(x_test)
    print("***** Scikit-Learn Logistic Regression Model *****")
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

compare_own_logistic_regression_model_with_scikit_version()

### Logistic Regression Class Boundaries

In [ ]:
def plot_decision_boundaries(lr, Xin, y, title=""):
    new_x = Xin[['confirmed_cases', 'deaths_per_case']]
    Xb = copy.deepcopy(new_x)
    lr.fit(Xb, y)  # train only on two features

    h = 0.01
    # create a mesh to plot in
    x_min, x_max = Xb.avg_glucose_level.min() - 1, Xb.avg_glucose_level.max() + 1
    y_min, y_max = Xb.bmi.min() - 1, Xb.bmi.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    # get prediction values
    Z = lr.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.5)

    # Plot also the training points
    plt.scatter(Xb.avg_glucose_level, Xb.bmi, c=y, cmap=plt.cm.Paired)
    plt.xlabel("Confirmed Cases")
    plt.ylabel("Deaths per Case")
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.xticks(())
    plt.yticks(())
    plt.title(title)
    plt.show()


mc_lr_model = MCLogisticRegression(eta=0.1, iterations=500)
plot_decision_boundaries(
    mc_lr_model, x_train, y_train, "Logistic Regression Class Boundaries (User-Defined)"
)

In [ ]:
lr_model = LogisticRegression(solver='liblinear')
plot_decision_boundaries(
    lr_model, x_train, y_train, "Logistic Regression Class Boundaries (Sklearn)"
)

### Regularized Binary Logistic Regression

In [ ]:
class RegularizedBinaryLogisticRegression(VectorBinaryLogisticRegression):
    # extend init functions
    def __init__(self, C=0.0, **kwargs):
        # need to add to the original initializer
        self.C = C
        # but keep other keywords
        super().__init__(**kwargs)  # call parent initializer

    def __str__(self):
        if hasattr(self, "w_"):
            return (
                "Regularized Binary Logistic Regression Object with coefficients:\n"
                + str(self.w_)
            )  # is we have trained the object
        else:
            return "Untrained Regularized Binary Logistic Logistic Regression Object"

    # extend previous class to change functionality
    def _get_gradient(self, X, y, reg="l1+l2"):
        # call get gradient from previous class
        gradient = super()._get_gradient(X, y)


        if reg == "l1":
            # add in regularization (to all except bias term)
            gradient[1:] += -2 * np.sign(self.w_[1:]) * self.C
            return gradient
        
        elif reg == "l2":
            # add in regularization (to all except bias term)
            gradient[1:] += -2 * self.w_[1:] * self.C
            return gradient
        
        if reg == "l1+l2":
            # add in regularization (to all except bias term)
            l1gradient = -2 * np.sign(self.w_[1:]) * self.C
            l2gradient = -2 * self.w_[1:] * self.C
            gradient[1:] += l1gradient
            gradient[1:] += l2gradient
            return gradient

In [ ]:
class RegularizedLogisticRegression(MCLogisticRegression):
    def __init__(self, C=0.0, **kwargs):
        # need to add to the original initializer
        self.C = C
        # but keep other keywords
        super().__init__(**kwargs)  # call parent initializer
    
    def __str__(self):
        if hasattr(self, "w_"):
            return (
                "Regularized Logistic Regression Object with coefficients:\n"
                + str(self.w_)
            )  # is we have trained the object
        else:
            return "Regularized Logistic Regression Object"

    def fit(self, X, y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y)  # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = []  # will fill this array with binary classifiers

        for i, yval in enumerate(self.unique_):  # for each unique value
            y_binary = y == yval  # create a binary problem
            # train the binary classifier for this class
            # now this has regularization built into it
            blr = RegularizedBinaryLogisticRegression(
                eta=self.eta, iterations=self.iters, C=self.C
            )
            blr.fit(X, y_binary)
            # add the trained classifier to the list
            self.classifiers_.append(blr)

        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T

In [ ]:
def run_regularized_binary_logistic_regression(**kwargs):
    rblr = RegularizedBinaryLogisticRegression(**kwargs)
    rblr.fit(x_train, y_train)
    y_pred = rblr.predict(x_test)
    
    print(f"Shape: {rblr.w_.shape}")
    print(rblr)
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

run_regularized_binary_logistic_regression(C=0.0, eta=0.1, iterations=1000)  

In [ ]:
def run_regularized_logistic_regression(**kwargs):
    rlr = RegularizedLogisticRegression(**kwargs)
    rlr.fit(x_train, y_train)
    y_pred = rlr.predict(x_test)
    
    print(f"Shape: {rlr.w_.shape}")
    print(rlr)
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

run_regularized_logistic_regression(C=0.0, eta=0.1, iterations=1000)

### Linear Regression Using Stochastic Gradient Descent

In [ ]:
class StochasticLogisticRegression:
    def __init__(self, eta=0.005, iterations=20, C=0.001):
        self.eta = eta
        self.iters = iterations
        self.C = C

    def __str__(self):
        if hasattr(self, "w_"):
            return (
                "Stochastic Logistic Regression Object with coefficients:\n"
                + str(self.w_)
            )  # is we have trained the object
        else:
            return "Untrained Stochastic Logistic Regression Object"

    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0], 1)), X))  # add bias term

    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta)  # 1/(1+np.exp(-theta))

    # vectorized gradient calculation with regularization using L2 Norm
    def _get_gradient(self, X, y):
        idx = np.random.randint(0, len(y)) # Grab random instance
        # idx = int(np.random.rand() * len(y))  # grab random instance

        y_val = y[idx]
        x_val = self.predict_proba(X[idx], add_bias=False)
        ydiff = y_val - x_val  # get y difference (now scalar)

        gradient = (
            X[idx] * ydiff[:, np.newaxis]
        )  # make ydiff a column vector and multiply through
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C

        return gradient

    def predict_proba(self, X, add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_)  # return the probability y=1

    def predict(self, X):
        return self.predict_proba(X) > 0.5  # return the actual prediction

    def fit(self, X, y):
        X = X.to_numpy()
        y = y.to_numpy()
        Xb = self._add_bias(X)  # add bias term
        num_samples, num_features = Xb.shape

        self.w_ = np.zeros((num_features, 1))  # init weight vector to zeros

        # for as many as the max iterations
        for ctr, _ in enumerate(range(self.iters)):
            gradient = self._get_gradient(Xb, y)
            self.w_ += gradient * self.eta  # multiply by learning rate

In [ ]:
def run_stochastic_gradient_descent(**kwargs):
    slr = StochasticLogisticRegression(**kwargs)
    slr.fit(x_train, y_train)
    y_pred = slr.predict(x_test)
    
    print(f"Shape: {slr.w_.shape}")
    print(slr)
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

run_stochastic_gradient_descent(C=0.0001, eta=0.005, iterations=8000)

### Steepest Gradient Descent

In [ ]:
class SteepestGradientDescentLogisticRegression:
    def __init__(self, eta=0.005, iterations=20, C=0.001):
        self.eta = eta
        self.iters = iterations
        self.C = C

    def __str__(self):
        if hasattr(self, "w_"):
            return (
                "Steepest Gradient Descent Logistic Regression Object with coefficients:\n"
                + str(self.w_)
            )  # is we have trained the object
        else:
            return "Untrained Steepest Gradient Descent Logistic Regression Object"

    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0], 1)), X))  # add bias term

    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta)  # 1/(1+np.exp(-theta))

    # vectorized gradient calculation with regularization using L2 Norm
    def _get_gradient(self, X, y):
        idx = np.random.randint(0, len(y)) # Grab random instance
        # idx = int(np.random.rand() * len(y))  # grab random instance

        y_val = y[idx]
        x_val = self.predict_proba(X[idx], add_bias=False)
        ydiff = y_val - x_val  # get y difference (now scalar)

        gradient = (
            X[idx] * ydiff[:, np.newaxis]
        )  # make ydiff a column vector and multiply through
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C

        return gradient

    def predict_proba(self, X, add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_)  # return the probability y=1

    def predict(self, X):
        return self.predict_proba(X) > 0.5  # return the actual prediction

    def fit(self, X, y):
        X = X.to_numpy()
        y = y.to_numpy()
        Xb = self._add_bias(X)  # add bias term
        num_samples, num_features = Xb.shape

        # self.w_ = np.zeros((num_features, 1))  # init weight vector to zeros
        self.w_ = np.zeros(num_features)

        # for as many as the max iterations
        for ctr, _ in enumerate(range(self.iters)):
            # gradient = self._get_gradient(Xb, y)
            # self.w_ += gradient * self.eta  # multiply by learning rate

            # Calculate the predicted values
            z = np.dot(Xb, self.w_)
            h = 1 / (1 + np.exp(-z))
            # Calculate the error
            error = h - y
            # Calculate the gradient
            gradient = np.dot(Xb.T, error) / num_samples
            # Update the parameters
            self.w_ = self.w_ - self.eta * gradient

In [ ]:
def run_steepest_gradient_descent(**kwargs):
    sgdlr = SteepestGradientDescentLogisticRegression(**kwargs)
    sgdlr.fit(x_train, y_train)
    y_pred = sgdlr.predict(x_test)
    
    print(f"Shape: {sgdlr.w_.shape}")
    print(sgdlr)
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

run_steepest_gradient_descent(C=0.0001, eta=0.005, iterations=8000)

### Hessian Binary Logistic Regression

In [ ]:
class HessianLogisticRegression:
    def __init__(self, eta=0.005, iterations=20, C=0.001):
        self.eta = eta
        self.iters = iterations
        self.C = C

    def __str__(self):
        if hasattr(self, "w_"):
            return (
                "Hessian Binary Logistic Regression Object with coefficients:\n"
                + str(self.w_)
            )  # is we have trained the object
        else:
            return "Untrained Hessian Binary Logistic Regression Object"

    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0], 1)), X))  # add bias term

    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta)  # 1/(1+np.exp(-theta))

    # vectorized gradient calculation with regularization using L2 Norm
    def _get_gradient(self, X, y):
        g = self.predict_proba(
            X, add_bias=False
        ).ravel()  # get sigmoid value for all classes
        hessian = (
            X.T @ np.diag(g * (1 - g)) @ X - 2 * self.C
        )  # calculate the hessian

        ydiff = y - g  # get y difference
        gradient = np.sum(
            X * ydiff[:, np.newaxis], axis=0
        )  # make ydiff a column vector and multiply through
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C

        return np.linalg.pinv(hessian) @ gradient

    def predict_proba(self, X, add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_)  # return the probability y=1

    def predict(self, X):
        return self.predict_proba(X) > 0.5  # return the actual prediction

    def fit(self, X, y):
        X = X.to_numpy()
        y = y.to_numpy()
        Xb = self._add_bias(X)  # add bias term
        num_samples, num_features = Xb.shape

        self.w_ = np.zeros((num_features, 1))  # init weight vector to zeros

        # for as many as the max iterations
        for ctr, _ in enumerate(range(self.iters)):
            gradient = self._get_gradient(Xb, y)
            self.w_ += gradient * self.eta  # multiply by learning rate

In [ ]:
def run_hessian_binary_logistic_regression(**kwargs):
    hblr = HessianLogisticRegression(**kwargs)
    hblr.fit(x_train, y_train)
    y_pred = hblr.predict(x_test)
    
    print(f"Shape: {hblr.w_.shape}")
    print(hblr)
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

run_hessian_binary_logistic_regression(C=0.0001, eta=0.005, iterations=600)

### Hessian Binary Logistic Regression (Rank Two Approximation) From Scratch

In [ ]:
class HessianLogisticRegressionRankTwo:
    def __init__(self, eta=0.005, iterations=20, C=0.001):
        self.eta = eta
        self.iters = iterations
        self.C = C

    def __str__(self):
        if hasattr(self, "w_"):
            return (
                "Hessian Binary Logistic Regression BFGS Object with coefficients:\n"
                + str(self.w_)
            )  # is we have trained the object
        else:
            return "Untrained Hessian Binary Logistic Regression BFGS Object"

    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0], 1)), X))  # add bias term

    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta)  # 1/(1+np.exp(-theta))

    @staticmethod
    def objective_function(w, X, y, C):
        g = expit(X @ w)
        # invert this because scipy minimizes, but we derived all formulas for maximzing
        return (
            -np.sum(np.ma.log(g[y == 1]))
            - np.sum(np.ma.log(1 - g[y == 0]))
            + C * sum(w**2)
        )
        # -np.sum(y*np.log(g)+(1-y)*np.log(1-g))

    @staticmethod
    def objective_gradient(w, X, y, C):
        g = expit(X @ w)
        ydiff = y - g  # get y difference
        gradient = np.mean(X * ydiff[:, np.newaxis], axis=0)
        gradient = gradient.reshape(w.shape)
        gradient[1:] += -2 * w[1:] * C
        return -gradient

    @staticmethod
    def backtracking_line_search(f, grad_f, x, p, *args, alpha=1, beta=0.5):
        """Perform a backtracking line search to find a step size that
        satisfies the Armijo condition.

        Parameters
        ----------
        f : callable
            Objective function to minimize.
        grad_f : callable
            Gradient function of the objective function.
        x : array-like
            Current point in the search space.
        p : array-like
            Search direction.
        args : tuple
            Extra arguments to pass to the objective and gradient functions.
        alpha : float, optional
            Initial step size.
        beta : float, optional
            Factor by which to decrease the step size at each iteration.

        Returns
        -------
        float
            Step size that satisfies the Armijo condition.
        """
        while f(x + alpha * p, *args) > f(x, *args) + alpha * beta * np.dot(
            grad_f(x, *args).T, p
        ):
            alpha *= beta
        return alpha

    def bfgs(
        self, f, x0, grad_f, args=(), epsilon=1e-5, max_iterations=10, verbose=False
    ):
        """Minimize a function using the BFGS algorithm.

        Parameters
        ----------
        f : callable
            Objective function to minimize.
        x0 : array-like
            Initial guess for the solution.
        grad_f : callable
            Gradient function of the objective function.
        args : tuple, optional
            Extra arguments to pass to the objective and gradient functions.
        epsilon : float, optional
            Tolerance for the norm of the gradient.
        max_iterations : int, optional
            Maximum number of iterations to perform.
        verbose : bool, optional
            Whether to print information about the optimization process.

        Returns
        -------
        x : ndarray
            The solution found by the optimization algorithm.
        """

        x = x0
        fx = f(x, *args)
        grad_fx = grad_f(x, *args)
        H = np.eye(x0.shape[0])

        for i in range(max_iterations):
            if np.linalg.norm(grad_fx) < epsilon:
                if verbose:
                    print(f"Optimization converged in {i} iterations.")
                return x

            p = -np.dot(H, grad_fx)
            alpha = self.backtracking_line_search(f, grad_f, x, p, *args)
            x_next = x + alpha * p
            s = x_next - x
            y = grad_f(x_next, *args) - grad_fx
            rho = 1 / np.dot(y.T, s)
            H = (np.eye(x.shape[0]) - rho * np.outer(s, y)) @ H @ (
                np.eye(x.shape[0]) - rho * np.outer(y, s)
            ) + rho * np.outer(s, s)
            x = x_next
            fx = f(x, *args)
            grad_fx = grad_f(x, *args)

            if verbose and i % 10 == 0:
                print(
                    f"Iteration {i}: f(x)={fx}, ||grad f(x)||={np.linalg.norm(grad_fx)}"
                )

        if verbose:
            print(f"Optimization stopped after {max_iterations} iterations.")
        return x

    def predict_proba(self, X, add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_)  # return the probability y=1

    def predict(self, X):
        return self.predict_proba(X) > 0.5  # return the actual prediction

    def fit(self, X, y):
        Xb = self._add_bias(X)  # add bias term
        num_samples, num_features = Xb.shape

        # Run bfgs here....
        self.w_ = self.bfgs(
            self.objective_function,
            np.zeros((num_features,)),
            self.objective_gradient,
            args=(Xb, y, self.C),
            epsilon=1e-3,
            max_iterations=self.iters,
            verbose=True,
        )

        self.w_ = self.w_.reshape((num_features, 1))

In [ ]:
def run_hessian_binary_logistic_regression_rank_two(**kwargs):
    hblr2 = HessianLogisticRegressionRankTwo(**kwargs)
    hblr2.fit(x_train, y_train)
    y_pred = hblr2.predict(x_test)
    
    print(f"Shape: {hblr2.w_.shape}")
    print(hblr2)
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

run_hessian_binary_logistic_regression_rank_two(C=0.0001, eta=0.005, iterations=30)

### Hessian Binary Logistic Regression Rank Two (Scipy Version)

In [ ]:
class HessianLogisticRegressionBFGS:
    def __init__(self, eta=0.005, iterations=20, C=0.001):
        self.eta = eta
        self.iters = iterations
        self.C = C

    def __str__(self):
        if hasattr(self, "w_"):
            return (
                "Hessian Binary Logistic Regression BFGS Object with coefficients:\n"
                + str(self.w_)
            )  # is we have trained the object
        else:
            return "Untrained Hessian Binary Logistic Regression BFGS Object"

    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0], 1)), X))  # add bias term

    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta)  # 1/(1+np.exp(-theta))

    @staticmethod
    def objective_function(w, X, y, C):
        g = expit(X @ w)
        # invert this because scipy minimizes, but we derived all formulas for maximzing
        return (
            -np.sum(np.ma.log(g[y == 1]))
            - np.sum(np.ma.log(1 - g[y == 0]))
            + C * sum(w**2)
        )
        # -np.sum(y*np.log(g)+(1-y)*np.log(1-g))

    @staticmethod
    def objective_gradient(w, X, y, C):
        g = expit(X @ w)
        ydiff = y - g  # get y difference
        gradient = np.mean(X * ydiff[:, np.newaxis], axis=0)
        gradient = gradient.reshape(w.shape)
        gradient[1:] += -2 * w[1:] * C
        return -gradient

    def predict_proba(self, X, add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_)  # return the probability y=1

    def predict(self, X):
        return self.predict_proba(X) > 0.5  # return the actual prediction

    def fit(self, X, y):
        Xb = self._add_bias(X)  # add bias term
        num_samples, num_features = Xb.shape

        self.w_ = scipy.optimize.fmin_bfgs(
            self.objective_function,  # what to optimize
            np.zeros((num_features, 1)),  # starting point
            fprime=self.objective_gradient,  # gradient function
            args=(
                Xb,
                y,
                self.C,
            ),  # extra args for gradient and objective function
            gtol=1e-03,  # stopping criteria for gradient, |v_k|
            maxiter=self.iters,  # stopping criteria iterations
            disp=False,
        )

        self.w_ = self.w_.reshape((num_features, 1))

In [ ]:
def run_hessian_binary_logistic_regression_bfgs(**kwargs):
    hblr_bfgs = HessianLogisticRegressionBFGS(**kwargs)
    hblr_bfgs.fit(x_train, y_train)
    y_pred = hblr_bfgs.predict(x_test)
    
    print(f"Shape: {hblr_bfgs.w_.shape}")
    print(hblr_bfgs)
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

run_hessian_binary_logistic_regression_bfgs(C=0.001, eta=0.005, iterations=3)

### BFGS and Newton's Method for Multiclass Logistic Regression

In [ ]:
class MultiClassLogisticRegression:
    def __init__(self, eta, iterations=20, 
                 C=0.0001, 
                 solver=HessianLogisticRegressionBFGS):
        self.eta = eta
        self.iters = iterations
        self.C = C
        self.solver = solver
        self.classifiers_ = []
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.sort(np.unique(y)) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = []
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = np.array(y==yval).astype(int) # create a binary problem
            # train the binary classifier for this class
            
            hblr = self.solver(eta=self.eta,iterations=self.iters,C=self.C)
            hblr.fit(X,y_binary)

            # add the trained classifier to the list
            self.classifiers_.append(hblr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        
    def predict_proba(self,X):
        probs = []
        for hblr in self.classifiers_:
            probs.append(hblr.predict_proba(X).reshape((len(X),1))) # get probability for each classifier
        
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        return np.argmax(self.predict_proba(X),axis=1) # take argmax along row

In [ ]:
def run_hessian_binary_logistic_regression_bfgs_and_newton(**kwargs):
    hblr_bfgs = MultiClassLogisticRegression(**kwargs)
    hblr_bfgs.fit(x_train, y_train)
    y_pred = hblr_bfgs.predict(x_test)
    
    print(f"Shape: {hblr_bfgs.w_.shape}")
    print(hblr_bfgs)
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")

run_hessian_binary_logistic_regression_bfgs(C=0.001, eta=0.005, iterations=5)

### Custom Logistic Regression Model Selector

[6] https://www.kaggle.com/code/junkal/selecting-the-best-regression-model

In [ ]:
class ModelSelector:
    models = [
        "Binary",
        "Line Search",
        "Vectorized Binary",
        "Multi-Class",  # One-Versus-All
        "L1 Regularized",
        "L2 Regularized",
        "L1+L2 Regularized",
        "Regularized Multi-Class",
        "Stochastic",
        "Steepest Gradient",
        "Hessian",
        "Hessian Rank Two",
        "Hessian Scipy",
        "Hessian BFGS+Newton Multi-Class",
    ]

    def __init__(
        self,
        x_train: Optional[pd.DataFrame] = None,
        y_train: Optional[pd.DataFrame] = None,
        x_test: Optional[pd.DataFrame] = None,
        y_test: Optional[pd.DataFrame] = None,
    ):
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.model_name = None
        self.results = []
    
    def model_accuracy(self, model):
        model.fit(self.x_train, self.y_train)
        predictions = model.predict(self.x_test)
        return accuracy_score(self.y_test, predictions)

    def print_models_scores(self):
        print(f"|{''.center(81,'=')}|")
        print(f"|{'MODEL'.center(40, ' ')}|{'ACCURACY'.center(40, ' ')}|")
        print(f"|{''.center(81,'=')}|")
        for name, accuracy in self.results:
            print(f"|{name.center(40, ' ')}|{str(round(accuracy, 4)).center(40, ' ')}|")
        print(f"|{''.center(81,'=')}|")

    def select_model(self, model_name, *args, **kwargs):
        if model_name == "Binary":
            model = self.BinaryLogisticRegression(*args, **kwargs)
        elif model_name == "Line Search":
            model = self.LineSearchLogisticRegression(*args, **kwargs)
        elif model_name == "Vectorized Binary":
            model = self.VectorBinaryLogisticRegression(*args, **kwargs)
        elif model_name == "Multi-Class" or model_name == "One-Versus-All":
            model = self.MCLogisticRegression(*args, **kwargs)
        elif model_name == "L1 Regularized":
            model = self.L1RegularizedBinaryLogisticRegression(*args, **kwargs)
        elif model_name == "L2 Regularized":
            model = self.L2RegularizedBinaryLogisticRegression(*args, **kwargs)
        elif model_name == "L1+L2 Regularized":
            model = self.L1AndL2RegularizedBinaryLogisticRegression(*args, **kwargs)
        elif model_name == "Regularized Multi-Class":
            model = self.MCRegularizedLogisticRegression(*args, **kwargs)
        elif model_name == "Stochastic":
            model = self.StochasticLogisticRegression(*args, **kwargs)
        elif model_name == "Steepest Gradient":
            model = self.SteepestGradientDescentLogisticRegression(*args, **kwargs)
        elif model_name == "Hessian":
            model = self.HessianLogisticRegression(*args, **kwargs)
        elif model_name == "Hessian Rank Two":
            model = self.HessianLogisticRegressionRankTwo(*args, **kwargs)
        elif model_name == "Hessian Scipy":
            model = self.HessianLogisticRegressionBFGS(*args, **kwargs)
        elif model_name == "Hessian BFGS+Newton Multi-Class":
            model = self.MultiClassLogisticRegression(*args, **kwargs)
        else:
            print("Invalid model")

        accuracy = self.model_accuracy(model)
        
        self.model_name = model_name
        self.results.append((model_name, accuracy))

        return model

    class BinaryLogisticRegressionBase:
        def __init__(self, eta, iterations=20, C: Optional[float] = None):
            self.eta = eta
            self.iters = iterations

        def __str__(self):
            return "Base Binary Logistic Regression Object, Not Trainable"

        # convenience, private and static:
        @staticmethod
        def _sigmoid(theta):
            return 1 / (1 + np.exp(-theta))

        @staticmethod
        def _add_intercept(X):
            return np.hstack((np.ones((X.shape[0], 1)), X))  # add bias term

        def _add_bias(self, X):
            return self._add_intercept(X)

        def predict_proba(self, X, add_intercept=True):
            # add bias term if requested
            Xb = self._add_intercept(X) if add_intercept else X
            return self._sigmoid(Xb @ self.w_)  # return the probability y=1

        def predict(self, X):
            return self.predict_proba(X) > 0.5  # return the actual prediction

    class BinaryLogisticRegression(BinaryLogisticRegressionBase):
        def __str__(self):
            if hasattr(self, "w_"):
                return "Binary Logistic Regression Object with coefficients:\n" + str(
                    self.w_
                )  # is we have trained the object
            else:
                return "Untrained Binary Logistic Regression Object"

        def _get_gradient(self, X, y):
            # programming \sum_i (yi-g(xi))xi
            gradient = np.zeros(self.w_.shape)  # set gradient to zero
            for (xi, yi) in zip(X, y):
                # the actual update inside of sum
                gradi = (yi - self.predict_proba(xi, add_intercept=False)) * xi
                # reshape to be column vector and add to gradient
                gradient += gradi.reshape(self.w_.shape)

            return gradient / float(len(y))

        def fit(self, X, y):
            Xb = self._add_intercept(X)  # add bias term
            num_samples, num_features = Xb.shape

            # init weight vector to zeros
            self.w_ = np.zeros((num_features, 1))

            # for as many as the max iterations
            for _ in range(self.iters):
                gradient = self._get_gradient(Xb, y)
                self.w_ += gradient * self.eta  # multiply by learning rate

    class LineSearchLogisticRegression(BinaryLogisticRegression):

        # define custom line search for problem
        def __init__(self, iterations=0.0, C=0.001, **kwargs):
            self.iterations = iterations
            self.C = C
            # but keep other keywords
            super().__init__(**kwargs)  # call parent initializer

        # this defines the function with the first input to be optimized
        # therefore eta will be optimized, with all inputs constant
        @staticmethod
        def objective_function(eta, X, y, w, grad, C):
            wnew = w - grad * eta
            g = expit(X @ wnew)
            # the line search is looking for minimization, so take the negative of l(w)
            return (
                -np.sum(np.ma.log(g[y == 1]))
                - np.ma.sum(np.log(1 - g[y == 0]))
                + C * sum(wnew**2)
            )

        def fit(self, X, y):
            Xb = self._add_bias(X)  # add bias term
            num_samples, num_features = Xb.shape

            self.w_ = np.zeros((num_features, 1))  # init weight vector to zeros

            # for as many as the max iterations
            for _ in range(self.iters):
                gradient = -self._get_gradient(Xb, y)
                # minimization is in opposite direction

                # do line search in gradient direction, using scipy function
                opts = {
                    "maxiter": self.iterations
                } 
                res = scipy.optimize.minimize_scalar(
                    self.objective_function,  # objective function to optimize
                    bounds=(0, self.eta * 10),  # bounds to optimize
                    args=(
                        Xb,
                        y,
                        self.w_,
                        gradient,
                        self.C,
                    ),  
                    method="bounded",  # bounded optimization for speed
                    options=opts,
                )  # set max iterations

                eta = res.x  # get optimal learning rate
                self.w_ -= gradient * eta  # set new function values

    class VectorBinaryLogisticRegression(BinaryLogisticRegression):
        def __str__(self):
            if hasattr(self, "w_"):
                return (
                    "Vector Binary Logistic Regression Object with coefficients:\n"
                    + str(self.w_)
                )  # is we have trained the object
            else:
                return "Untrained Vector Binary Logistic Regression Object"

        # inherit from our previous class to get same functionality
        @staticmethod
        def _sigmoid(theta):
            # increase stability, redefine sigmoid operation
            return expit(theta)  # 1/(1+np.exp(-theta))

        # but overwrite the gradient calculation
        def _get_gradient(self, X, y):
            ydiff = (
                y - self.predict_proba(X, add_intercept=False).ravel()
            )  # get y difference
            gradient = np.mean(
                X * ydiff[:, np.newaxis], axis=0
            )  # make ydiff a column vector and multiply through

            return gradient.reshape(self.w_.shape)

    class MCLogisticRegression:
        def __init__(self, eta, iterations=20, **kwargs):
            self.eta = eta
            self.iters = iterations

        def __str__(self):
            if hasattr(self, "w_"):
                return (
                    "MultiClass Logistic Regression Object with coefficients:\n"
                    + str(self.w_)
                )
            else:
                return "Untrained MultiClass Logistic Regression Object"

        def fit(self, X, y):
            num_samples, num_features = X.shape
            self.unique_ = np.unique(y)  # get each unique class value
            num_unique_classes = len(self.unique_)
            self.classifiers_ = []  # will fill this array with binary classifiers

            for i, yval in enumerate(self.unique_):  # for each unique value
                y_binary = y == yval  # create a binary problem
                # train the binary classifier for this class
                blr = VectorBinaryLogisticRegression(self.eta, self.iters)
                blr.fit(X, y_binary)
                # add the trained classifier to the list
                self.classifiers_.append(blr)

            # save all the weights into one matrix, separate column for each class
            self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T

        def predict_proba(self, X):
            probs = []
            for blr in self.classifiers_:
                probs.append(
                    blr.predict_proba(X)
                )  # get probability for each classifier

            return np.hstack(probs)  # make into single matrix

        def predict(self, X):
            return self.unique_[
                np.argmax(self.predict_proba(X), axis=1)
            ]  # take argmax along row

    class L1RegularizedBinaryLogisticRegression(VectorBinaryLogisticRegression):
        # extend init functions
        def __init__(self, C=0.0, **kwargs):
            # need to add to the original initializer
            self.C = C
            # but keep other keywords
            super().__init__(**kwargs)  # call parent initializer

        def __str__(self):
            if hasattr(self, "w_"):
                return (
                    "Regularized Binary Logistic Regression Object with coefficients:\n"
                    + str(self.w_)
                )  # is we have trained the object
            else:
                return (
                    "Untrained Regularized Binary Logistic Logistic Regression Object"
                )

        # extend previous class to change functionality
        def _get_gradient(self, X, y):
            # call get gradient from previous class
            gradient = super()._get_gradient(X, y)
            gradient[1:] += -2 * np.sign(self.w_[1:]) * self.C
            return gradient

    class L2RegularizedBinaryLogisticRegression(VectorBinaryLogisticRegression):
        # extend init functions
        def __init__(self, C=0.0, **kwargs):
            # need to add to the original initializer
            self.C = C
            # but keep other keywords
            super().__init__(**kwargs)  # call parent initializer

        def __str__(self):
            if hasattr(self, "w_"):
                return (
                    "Regularized Binary Logistic Regression Object with coefficients:\n"
                    + str(self.w_)
                )  # is we have trained the object
            else:
                return (
                    "Untrained Regularized Binary Logistic Logistic Regression Object"
                )

        # extend previous class to change functionality
        def _get_gradient(self, X, y):
            # call get gradient from previous class
            gradient = super()._get_gradient(X, y)
            gradient[1:] += -2 * self.w_[1:] * self.C
            return gradient

    class L1AndL2RegularizedBinaryLogisticRegression(VectorBinaryLogisticRegression):
        # extend init functions
        def __init__(self, C=0.0, **kwargs):
            # need to add to the original initializer
            self.C = C
            # but keep other keywords
            super().__init__(**kwargs)  # call parent initializer

        def __str__(self):
            if hasattr(self, "w_"):
                return (
                    "Regularized Binary Logistic Regression Object with coefficients:\n"
                    + str(self.w_)
                )  # is we have trained the object
            else:
                return (
                    "Untrained Regularized Binary Logistic Logistic Regression Object"
                )

        # extend previous class to change functionality
        def _get_gradient(self, X, y):
            # call get gradient from previous class
            gradient = super()._get_gradient(X, y)
            l1gradient = -2 * np.sign(self.w_[1:]) * self.C
            l2gradient = -2 * self.w_[1:] * self.C
            gradient[1:] += l1gradient
            gradient[1:] += l2gradient
            return gradient

    class MCRegularizedLogisticRegression(MCLogisticRegression):
        def __init__(self, C=0.0, **kwargs):
            # need to add to the original initializer
            self.C = C
            # but keep other keywords
            super().__init__(**kwargs)  # call parent initializer

        def __str__(self):
            if hasattr(self, "w_"):
                return (
                    "Regularized Logistic Regression Object with coefficients:\n"
                    + str(self.w_)
                )  # is we have trained the object
            else:
                return "Regularized Logistic Regression Object"

        def fit(self, X, y):
            num_samples, num_features = X.shape
            self.unique_ = np.unique(y)  # get each unique class value
            num_unique_classes = len(self.unique_)
            self.classifiers_ = []  # will fill this array with binary classifiers

            for i, yval in enumerate(self.unique_):  # for each unique value
                y_binary = y == yval  # create a binary problem
                # train the binary classifier for this class
                # now this has regularization built into it
                blr = RegularizedBinaryLogisticRegression(
                    eta=self.eta, iterations=self.iters, C=self.C
                )
                blr.fit(X, y_binary)
                # add the trained classifier to the list
                self.classifiers_.append(blr)

            # save all the weights into one matrix, separate column for each class
            self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T

    class StochasticLogisticRegression:
        def __init__(self, eta=0.005, iterations=20, C=0.001):
            self.eta = eta
            self.iters = iterations
            self.C = C

        def __str__(self):
            if hasattr(self, "w_"):
                return (
                    "Stochastic Logistic Regression Object with coefficients:\n"
                    + str(self.w_)
                )  # is we have trained the object
            else:
                return "Untrained Stochastic Logistic Regression Object"

        @staticmethod
        def _add_bias(X):
            return np.hstack((np.ones((X.shape[0], 1)), X))  # add bias term

        @staticmethod
        def _sigmoid(theta):
            # increase stability, redefine sigmoid operation
            return expit(theta)  # 1/(1+np.exp(-theta))

        # vectorized gradient calculation with regularization using L2 Norm
        def _get_gradient(self, X, y):
            idx = np.random.randint(0, len(y))  # Grab random instance
            # idx = int(np.random.rand() * len(y))  # grab random instance

            y_val = y[idx]
            x_val = self.predict_proba(X[idx], add_bias=False)
            ydiff = y_val - x_val  # get y difference (now scalar)

            gradient = (
                X[idx] * ydiff[:, np.newaxis]
            )  # make ydiff a column vector and multiply through
            gradient = gradient.reshape(self.w_.shape)
            gradient[1:] += -2 * self.w_[1:] * self.C

            return gradient

        def predict_proba(self, X, add_bias=True):
            # add bias term if requested
            Xb = self._add_bias(X) if add_bias else X
            return self._sigmoid(Xb @ self.w_)  # return the probability y=1

        def predict(self, X):
            return self.predict_proba(X) > 0.5  # return the actual prediction

        def fit(self, X, y):
            X = X.to_numpy()
            y = y.to_numpy()
            Xb = self._add_bias(X)  # add bias term
            num_samples, num_features = Xb.shape

            self.w_ = np.zeros((num_features, 1))  # init weight vector to zeros

            # for as many as the max iterations
            for ctr, _ in enumerate(range(self.iters)):
                gradient = self._get_gradient(Xb, y)
                self.w_ += gradient * self.eta  # multiply by learning rate

    class SteepestGradientDescentLogisticRegression:
        def __init__(self, eta=0.005, iterations=20, C=0.001):
            self.eta = eta
            self.iters = iterations
            self.C = C

        def __str__(self):
            if hasattr(self, "w_"):
                return (
                    "Steepest Gradient Descent Logistic Regression Object with coefficients:\n"
                    + str(self.w_)
                )  # is we have trained the object
            else:
                return "Untrained Steepest Gradient Descent Logistic Regression Object"

        @staticmethod
        def _add_bias(X):
            return np.hstack((np.ones((X.shape[0], 1)), X))  # add bias term

        @staticmethod
        def _sigmoid(theta):
            # increase stability, redefine sigmoid operation
            return expit(theta)  # 1/(1+np.exp(-theta))

        # vectorized gradient calculation with regularization using L2 Norm
        def _get_gradient(self, X, y):
            idx = np.random.randint(0, len(y))  # Grab random instance
            # idx = int(np.random.rand() * len(y))  # grab random instance

            y_val = y[idx]
            x_val = self.predict_proba(X[idx], add_bias=False)
            ydiff = y_val - x_val  # get y difference (now scalar)

            gradient = (
                X[idx] * ydiff[:, np.newaxis]
            )  # make ydiff a column vector and multiply through
            gradient = gradient.reshape(self.w_.shape)
            gradient[1:] += -2 * self.w_[1:] * self.C

            return gradient

        def predict_proba(self, X, add_bias=True):
            # add bias term if requested
            Xb = self._add_bias(X) if add_bias else X
            return self._sigmoid(Xb @ self.w_)  # return the probability y=1

        def predict(self, X):
            return self.predict_proba(X) > 0.5  # return the actual prediction

        def fit(self, X, y):
            X = X.to_numpy()
            y = y.to_numpy()
            Xb = self._add_bias(X)  # add bias term
            num_samples, num_features = Xb.shape

            # self.w_ = np.zeros((num_features, 1))  # init weight vector to zeros
            self.w_ = np.zeros(num_features)

            # for as many as the max iterations
            for ctr, _ in enumerate(range(self.iters)):
                # gradient = self._get_gradient(Xb, y)
                # self.w_ += gradient * self.eta  # multiply by learning rate

                # Calculate the predicted values
                z = np.dot(Xb, self.w_)
                h = 1 / (1 + np.exp(-z))
                # Calculate the error
                error = h - y
                # Calculate the gradient
                gradient = np.dot(Xb.T, error) / num_samples
                # Update the parameters
                self.w_ = self.w_ - self.eta * gradient

    class HessianLogisticRegression:
        def __init__(self, eta=0.005, iterations=20, C=0.001):
            self.eta = eta
            self.iters = iterations
            self.C = C

        def __str__(self):
            if hasattr(self, "w_"):
                return (
                    "Hessian Binary Logistic Regression Object with coefficients:\n"
                    + str(self.w_)
                )  # is we have trained the object
            else:
                return "Untrained Hessian Binary Logistic Regression Object"

        @staticmethod
        def _add_bias(X):
            return np.hstack((np.ones((X.shape[0], 1)), X))  # add bias term

        @staticmethod
        def _sigmoid(theta):
            # increase stability, redefine sigmoid operation
            return expit(theta)  # 1/(1+np.exp(-theta))

        # vectorized gradient calculation with regularization using L2 Norm
        def _get_gradient(self, X, y):
            g = self.predict_proba(
                X, add_bias=False
            ).ravel()  # get sigmoid value for all classes
            hessian = (
                X.T @ np.diag(g * (1 - g)) @ X - 2 * self.C
            )  # calculate the hessian

            ydiff = y - g  # get y difference
            gradient = np.sum(
                X * ydiff[:, np.newaxis], axis=0
            )  # make ydiff a column vector and multiply through
            gradient = gradient.reshape(self.w_.shape)
            gradient[1:] += -2 * self.w_[1:] * self.C

            return np.linalg.pinv(hessian) @ gradient

        def predict_proba(self, X, add_bias=True):
            # add bias term if requested
            Xb = self._add_bias(X) if add_bias else X
            return self._sigmoid(Xb @ self.w_)  # return the probability y=1

        def predict(self, X):
            return self.predict_proba(X) > 0.5  # return the actual prediction

        def fit(self, X, y):
            X = X.to_numpy()
            y = y.to_numpy()
            Xb = self._add_bias(X)  # add bias term
            num_samples, num_features = Xb.shape

            self.w_ = np.zeros((num_features, 1))  # init weight vector to zeros

            # for as many as the max iterations
            for ctr, _ in enumerate(range(self.iters)):
                gradient = self._get_gradient(Xb, y)
                self.w_ += gradient * self.eta  # multiply by learning rate

    class HessianLogisticRegressionRankTwo:
        def __init__(self, eta=0.005, iterations=20, C=0.001):
            self.eta = eta
            self.iters = iterations
            self.C = C

        def __str__(self):
            if hasattr(self, "w_"):
                return (
                    "Hessian Binary Logistic Regression BFGS Object with coefficients:\n"
                    + str(self.w_)
                )  # is we have trained the object
            else:
                return "Untrained Hessian Binary Logistic Regression BFGS Object"

        @staticmethod
        def _add_bias(X):
            return np.hstack((np.ones((X.shape[0], 1)), X))  # add bias term

        @staticmethod
        def _sigmoid(theta):
            # increase stability, redefine sigmoid operation
            return expit(theta)  # 1/(1+np.exp(-theta))

        @staticmethod
        def objective_function(w, X, y, C):
            g = expit(X @ w)
            # invert this because scipy minimizes, but we derived all formulas for maximzing
            return (
                -np.sum(np.ma.log(g[y == 1]))
                - np.sum(np.ma.log(1 - g[y == 0]))
                + C * sum(w**2)
            )
            # -np.sum(y*np.log(g)+(1-y)*np.log(1-g))

        @staticmethod
        def objective_gradient(w, X, y, C):
            g = expit(X @ w)
            ydiff = y - g  # get y difference
            gradient = np.mean(X * ydiff[:, np.newaxis], axis=0)
            gradient = gradient.reshape(w.shape)
            gradient[1:] += -2 * w[1:] * C
            return -gradient

        @staticmethod
        def backtracking_line_search(f, grad_f, x, p, *args, alpha=1, beta=0.5):
            """Perform a backtracking line search to find a step size that
            satisfies the Armijo condition.

            Parameters
            ----------
            f : callable
                Objective function to minimize.
            grad_f : callable
                Gradient function of the objective function.
            x : array-like
                Current point in the search space.
            p : array-like
                Search direction.
            args : tuple
                Extra arguments to pass to the objective and gradient functions.
            alpha : float, optional
                Initial step size.
            beta : float, optional
                Factor by which to decrease the step size at each iteration.

            Returns
            -------
            float
                Step size that satisfies the Armijo condition.
            """
            while f(x + alpha * p, *args) > f(x, *args) + alpha * beta * np.dot(
                grad_f(x, *args).T, p
            ):
                alpha *= beta
            return alpha

        def bfgs(
            self, f, x0, grad_f, args=(), epsilon=1e-5, max_iterations=10, verbose=False
        ):
            """Minimize a function using the BFGS algorithm.

            Parameters
            ----------
            f : callable
                Objective function to minimize.
            x0 : array-like
                Initial guess for the solution.
            grad_f : callable
                Gradient function of the objective function.
            args : tuple, optional
                Extra arguments to pass to the objective and gradient functions.
            epsilon : float, optional
                Tolerance for the norm of the gradient.
            max_iterations : int, optional
                Maximum number of iterations to perform.
            verbose : bool, optional
                Whether to print information about the optimization process.

            Returns
            -------
            x : ndarray
                The solution found by the optimization algorithm.
            """

            x = x0
            fx = f(x, *args)
            grad_fx = grad_f(x, *args)
            H = np.eye(x0.shape[0])

            for i in range(max_iterations):
                if np.linalg.norm(grad_fx) < epsilon:
                    if verbose:
                        print(f"Optimization converged in {i} iterations.")
                    return x

                p = -np.dot(H, grad_fx)
                alpha = self.backtracking_line_search(f, grad_f, x, p, *args)
                x_next = x + alpha * p
                s = x_next - x
                y = grad_f(x_next, *args) - grad_fx
                rho = 1 / np.dot(y.T, s)
                H = (np.eye(x.shape[0]) - rho * np.outer(s, y)) @ H @ (
                    np.eye(x.shape[0]) - rho * np.outer(y, s)
                ) + rho * np.outer(s, s)
                x = x_next
                fx = f(x, *args)
                grad_fx = grad_f(x, *args)

                if verbose and i % 10 == 0:
                    print(
                        f"Iteration {i}: f(x)={fx}, ||grad f(x)||={np.linalg.norm(grad_fx)}"
                    )

            if verbose:
                print(f"Optimization stopped after {max_iterations} iterations.")
            return x

        def predict_proba(self, X, add_bias=True):
            # add bias term if requested
            Xb = self._add_bias(X) if add_bias else X
            return self._sigmoid(Xb @ self.w_)  # return the probability y=1

        def predict(self, X):
            return self.predict_proba(X) > 0.5  # return the actual prediction

        def fit(self, X, y):
            Xb = self._add_bias(X)  # add bias term
            num_samples, num_features = Xb.shape

            # Run bfgs here....
            self.w_ = self.bfgs(
                self.objective_function,
                np.zeros((num_features,)),
                self.objective_gradient,
                args=(Xb, y, self.C),
                epsilon=1e-3,
                max_iterations=self.iters,
                verbose=False,
            )

            self.w_ = self.w_.reshape((num_features, 1))

    class HessianLogisticRegressionBFGS:
        def __init__(self, eta=0.005, iterations=20, C=0.001):
            self.eta = eta
            self.iters = iterations
            self.C = C

        def __str__(self):
            if hasattr(self, "w_"):
                return (
                    "Hessian Binary Logistic Regression BFGS Object with coefficients:\n"
                    + str(self.w_)
                )  # is we have trained the object
            else:
                return "Untrained Hessian Binary Logistic Regression BFGS Object"

        @staticmethod
        def _add_bias(X):
            return np.hstack((np.ones((X.shape[0], 1)), X))  # add bias term

        @staticmethod
        def _sigmoid(theta):
            # increase stability, redefine sigmoid operation
            return expit(theta)  # 1/(1+np.exp(-theta))

        @staticmethod
        def objective_function(w, X, y, C):
            g = expit(X @ w)
            # invert this because scipy minimizes, but we derived all formulas for maximzing
            return (
                -np.sum(np.ma.log(g[y == 1]))
                - np.sum(np.ma.log(1 - g[y == 0]))
                + C * sum(w**2)
            )
            # -np.sum(y*np.log(g)+(1-y)*np.log(1-g))

        @staticmethod
        def objective_gradient(w, X, y, C):
            g = expit(X @ w)
            ydiff = y - g  # get y difference
            gradient = np.mean(X * ydiff[:, np.newaxis], axis=0)
            gradient = gradient.reshape(w.shape)
            gradient[1:] += -2 * w[1:] * C
            return -gradient

        def predict_proba(self, X, add_bias=True):
            # add bias term if requested
            Xb = self._add_bias(X) if add_bias else X
            return self._sigmoid(Xb @ self.w_)  # return the probability y=1

        def predict(self, X):
            return self.predict_proba(X) > 0.5  # return the actual prediction

        def fit(self, X, y):
            Xb = self._add_bias(X)  # add bias term
            num_samples, num_features = Xb.shape

            self.w_ = scipy.optimize.fmin_bfgs(
                self.objective_function,  # what to optimize
                np.zeros((num_features, 1)),  # starting point
                fprime=self.objective_gradient,  # gradient function
                args=(
                    Xb,
                    y,
                    self.C,
                ),  # extra args for gradient and objective function
                gtol=1e-03,  # stopping criteria for gradient, |v_k|
                maxiter=self.iters,  # stopping criteria iterations
                disp=False,
            )

            self.w_ = self.w_.reshape((num_features, 1))

    class MultiClassLogisticRegression:
        def __init__(
            self, eta, iterations=20, C=0.0001, solver=HessianLogisticRegressionBFGS
        ):
            self.eta = eta
            self.iters = iterations
            self.C = C
            self.solver = solver
            self.classifiers_ = []
            # internally we will store the weights as self.w_ to keep with sklearn conventions

        def __str__(self):
            if hasattr(self, "w_"):
                return (
                    "MultiClass Logistic Regression Object with coefficients:\n"
                    + str(self.w_)
                )  # is we have trained the object
            else:
                return "Untrained MultiClass Logistic Regression Object"

        def fit(self, X, y):
            num_samples, num_features = X.shape
            self.unique_ = np.sort(np.unique(y))  # get each unique class value
            num_unique_classes = len(self.unique_)
            self.classifiers_ = []
            for i, yval in enumerate(self.unique_):  # for each unique value
                y_binary = np.array(y == yval).astype(int)  # create a binary problem
                # train the binary classifier for this class

                hblr = self.solver(eta=self.eta, iterations=self.iters, C=self.C)
                hblr.fit(X, y_binary)

                # add the trained classifier to the list
                self.classifiers_.append(hblr)

            # save all the weights into one matrix, separate column for each class
            self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T

        def predict_proba(self, X):
            probs = []
            for hblr in self.classifiers_:
                probs.append(
                    hblr.predict_proba(X).reshape((len(X), 1))
                )  # get probability for each classifier

            return np.hstack(probs)  # make into single matrix

        def predict(self, X):
            return np.argmax(self.predict_proba(X), axis=1)  # take argmax along row

In [ ]:
model_selector = ModelSelector(x_train, y_train, x_test, y_test)
model_names = model_selector.models
kwargs = {'eta': 0.005, 'iterations': 500, 'C': 0.001}
for model_name in model_names:
    model = model_selector.select_model(model_name, **kwargs)
model_selector.print_models_scores()

# Deployment

We will be using this section to discuss which implementation of logistic regression would you advise be used in a deployed machine learning model, your implementation or scikit-learn (or other third party)? Why?

# Conclusion

In summary, this section will be used to summarize the project goal and how we achieved it, and wrapping up any ideas presented back in the business understanding section.

# References

- [1] https://console.cloud.google.com/marketplace/details/usafacts-public-data/covid19-us-cases?filter=solution-type:dataset&filter=category:covid19&id=3eaff9c5-fbaf-47bb-a441-89db1e1395ab&project=still-nebula-398202

#### PLEASE COPY AND PASTE THE RESOURCES FROM THEIR CURRENT LOCATIONS AND PUT THEM HERE